In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, brier_score_loss, roc_curve
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    brier_score_loss,
    classification_report,
    roc_auc_score
)

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

In [2]:
X_train_smote_dropped =pd.read_csv('X_train_smote_dropped.csv').drop(columns=['Unnamed: 0'])
X_train_dropped = pd.read_csv('X_train_dropped.csv').drop(columns=['Unnamed: 0'])
X_test_dropped = pd.read_csv('X_test_dropped.csv').drop(columns=['Unnamed: 0']) 

y_train_smote = pd.read_csv('y_train_smote.csv')['fraudulent']
y_test = pd.read_csv('y_test.csv')['fraudulent']
y_train = pd.read_csv('y_train.csv')['fraudulent']

**LSTM Hyperparameter tuning using SMOTE Dataset**

In [3]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_smote_dropped)
X_test_scaled = scaler.transform(X_test_dropped)

X_train_reshaped = np.expand_dims(X_train_scaled, axis=1)  
X_test_reshaped = np.expand_dims(X_test_scaled, axis=1)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

#Building the LSTM model
def build_lstm_model(units_1, units_2, dropout_1, dropout_2, learning_rate):
    model = Sequential([
        Bidirectional(LSTM(units_1, activation='tanh', return_sequences=True), input_shape=(1, X_train_scaled.shape[1])),
        Dropout(dropout_1),
        LSTM(units_2, activation='tanh', return_sequences=False),
        Dropout(dropout_2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')  
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(
    model=build_lstm_model,
    verbose=0,
)

param_grid = {
    "model__units_1": [64, 128],
    "model__units_2": [32, 64],
    "model__dropout_1": [0.2, 0.3],
    "model__dropout_2": [0.3, 0.4],
    "model__learning_rate": [1e-2, 1e-3],
    "batch_size": [32, 64],
    "epochs": [10, 20],
}

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=10,
    scoring='roc_auc',
    cv=3,
    verbose=1
)

random_search.fit(X_train_reshaped, y_train_smote)

print("Best parameters found:", random_search.best_params_)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: 

Best parameters found: {'model__units_2': 64, 'model__units_1': 64, 'model__learning_rate': 0.01, 'model__dropout_2': 0.4, 'model__dropout_1': 0.2, 'epochs': 20, 'batch_size': 64}


In [5]:
best_params = random_search.best_params_
print(random_search.best_params_)


{'model__units_2': 64, 'model__units_1': 64, 'model__learning_rate': 0.01, 'model__dropout_2': 0.4, 'model__dropout_1': 0.2, 'epochs': 20, 'batch_size': 64}


In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

model = Sequential([
    Bidirectional(LSTM(best_params['model__units_1'], activation='tanh', return_sequences=True), input_shape=(1, X_train_reshaped.shape[2])),
    Dropout(best_params['model__dropout_1']),
    LSTM(best_params['model__units_2'], activation='tanh', return_sequences=False),
    Dropout(best_params['model__dropout_2']),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  
])

optimizer = Adam(learning_rate=best_params['model__learning_rate'])
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_smote),  
    y=y_train_smote  
)
class_weight_dict = dict(enumerate(class_weights))

history = model.fit(
    X_train_reshaped, y_train_smote,
    validation_data=(X_test_reshaped, y_test),
    epochs=best_params['epochs'],
    batch_size=best_params['batch_size'],
    class_weight=class_weight_dict,  
    callbacks=[lr_scheduler, early_stopping]
)


y_train_proba = model.predict(X_train_reshaped).flatten()
y_test_proba = model.predict(X_test_reshaped).flatten()

y_train_pred = (y_train_proba > 0.5).astype(int)
print("Training Metrics:")
print(classification_report(y_train_smote, y_train_pred))
print("ROC AUC Score:", roc_auc_score(y_train_smote, y_train_proba))
print("Brier Score:", brier_score_loss(y_train_smote, y_train_proba))

y_test_pred = (y_test_proba > 0.5).astype(int)
print("\nTest Metrics:")
print(classification_report(y_test, y_test_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_test_proba))
print("Brier Score:", brier_score_loss(y_test, y_test_proba))


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
426/426 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9404 - loss: 0.1399 - val_accuracy: 0.9835 - val_loss: 0.0758 - learning_rate: 0.0100
Epoch 2/20
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9960 - loss: 0.0150 - val_accuracy: 0.9874 - val_loss: 0.0692 - learning_rate: 0.0100
Epoch 3/20
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9979 - loss: 0.0062 - val_accuracy: 0.9843 - val_loss: 0.0848 - learning_rate: 0.0100
Epoch 4/20
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9985 - loss: 0.0053 - val_accuracy: 0.9799 - val_loss: 0.0883 - learning_rate: 0.0100
Epoch 5/20
423/426 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9986 - loss: 0.0066
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9986 - loss: 0.0066 - val_accuracy: 0.9871 - val_loss: 0.0906 - learning_rate: 0.0100
Epoch 6/20
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9995 - loss: 0.0016 - va

**LSTM Pseudolabelling using SMOTE Dataset**

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.metrics import roc_auc_score, brier_score_loss, classification_report

# Creating the LSTM model with the best parameters 
model = Sequential([
    Bidirectional(LSTM(best_params['model__units_1'], activation='tanh', return_sequences=True), input_shape=(1, X_train_reshaped.shape[2])),
    Dropout(best_params['model__dropout_1']),
    LSTM(best_params['model__units_2'], activation='tanh', return_sequences=False),
    Dropout(best_params['model__dropout_2']),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  
])

optimizer = Adam(learning_rate=best_params['model__learning_rate'])
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),  
    y=y_train_smote  
)
class_weight_dict = dict(enumerate(class_weights))

augmented_train_data = X_train_smote_dropped.values  

augmented_train_labels = y_train_smote  
augmented_train_labels = np.reshape(augmented_train_labels, (-1, 1))  

augmented_train_data_reshaped = augmented_train_data.reshape((augmented_train_data.shape[0], 1, augmented_train_data.shape[1]))

model.fit(augmented_train_data_reshaped, augmented_train_labels, 
          epochs=best_params['epochs'], 
          batch_size=best_params['batch_size'],
          class_weight=class_weight_dict, 
          callbacks=[lr_scheduler, early_stopping])


confidence_threshold = 0.9  
n_iterations = 20  

for i in range(n_iterations):
    print(f"\n--- Pseudolabeling Iteration {i+1} ---")

    X_test_reshaped = X_test_dropped.values.reshape((X_test_dropped.shape[0], 1, X_test_dropped.shape[1]))

    val_pred_proba = model.predict(X_test_reshaped)

    pseudolabels = np.where(val_pred_proba.max(axis=1) > confidence_threshold, val_pred_proba.argmax(axis=1), -1)
    
    pseudolabeled_data = X_test_dropped.values[pseudolabels != -1]
    pseudolabel_targets = pseudolabels[pseudolabels != -1]

    if len(pseudolabeled_data) == 0:
        print("No high-confidence pseudolabels found. Stopping iteration.")
        break

    augmented_train_data = np.vstack([X_train_smote_dropped, pseudolabeled_data])
    augmented_train_labels = np.concatenate([y_train_smote, pseudolabel_targets])

    augmented_train_labels = np.reshape(augmented_train_labels, (-1, 1))  

    augmented_train_data_reshaped = augmented_train_data.reshape((augmented_train_data.shape[0], 1, augmented_train_data.shape[1]))

    model.fit(augmented_train_data_reshaped, augmented_train_labels, 
              epochs=best_params['epochs'], 
              batch_size=best_params['batch_size'],
              class_weight=class_weight_dict, 
              callbacks=[lr_scheduler, early_stopping])

print("\n--- Final Model Evaluation ---")

X_train_reshaped = X_train_smote_dropped.values.reshape((X_train_smote_dropped.shape[0], 1, X_train_smote_dropped.shape[1]))

y_train_pred = (model.predict(X_train_reshaped) > 0.5).astype(int)  
y_train_proba = model.predict(X_train_reshaped)  

print("\nTraining Set Metrics:")
print(classification_report(y_train_smote, y_train_pred))
print("ROC AUC Score:", roc_auc_score(y_train_smote, y_train_proba))
print("Brier Score:", brier_score_loss(y_train_smote, y_train_proba))


X_test_reshaped = X_test_dropped.values.reshape((X_test_dropped.shape[0], 1, X_test_dropped.shape[1]))

y_test_pred = (model.predict(X_test_reshaped) > 0.5).astype(int)
y_test_proba = model.predict(X_test_reshaped)  

print("\nTest Set Metrics:")
print(classification_report(y_test, y_test_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_test_proba))
print("Brier Score:", brier_score_loss(y_test, y_test_proba))


augmented_train_labels shape: (27240, 1)
augmented_train_data shape: (27240, 586)
augmented_train_data_reshaped shape: (27240, 1, 586)
augmented_train_labels shape: (27240, 1)
Epoch 1/20


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


426/426 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9439 - loss: 0.1477 - learning_rate: 0.0100
Epoch 2/20
 36/426 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9967 - loss: 0.0197

/opt/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/callback_list.py:151: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate
  current = self.get_monitor_value(logs)


426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9952 - loss: 0.0188 - learning_rate: 0.0100
Epoch 3/20
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9971 - loss: 0.0090 - learning_rate: 0.0100
Epoch 4/20
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9982 - loss: 0.0061 - learning_rate: 0.0100
Epoch 5/20
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9986 - loss: 0.0045 - learning_rate: 0.0100
Epoch 6/20
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9992 - loss: 0.0039 - learning_rate: 0.0100
Epoch 7/20
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9980 - loss: 0.0057 - learning_rate: 0.0100
Epoch 8/20
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9993 - loss: 0.0020 - learning_rate: 0.0100
Epoch 9/20
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9983 - loss: 0.0042 - learning_rate: 0.0100
Epoch 10/20
426/426 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9993 - loss: 0.0023 - learning_rate: 0.0100
Epoch 11/20
426/426 

**LSTM using Best Hyperparameters on Non-SMOTE Dataset**

In [20]:
scaler_non_smote = MinMaxScaler()
X_train_scaled_non_smote = scaler_non_smote.fit_transform(X_train_dropped)  
X_test_scaled_non_smote = scaler_non_smote.transform(X_test_dropped)  

X_train_reshaped_non_smote = np.expand_dims(X_train_scaled_non_smote, axis=1)
X_test_reshaped_non_smote = np.expand_dims(X_test_scaled_non_smote, axis=1)

print(f"X_train_reshaped_non_smote shape: {X_train_reshaped_non_smote.shape}")
print(f"y_train shape: {y_train.shape}")

X_train_reshaped_non_smote shape: (14303, 1, 586)
y_train shape: (14303,)


In [18]:
if len(y_train.shape) > 1:
    y_train = np.ravel(y_train)

print(y_train.shape)  



(14303,)


In [25]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import classification_report, roc_auc_score, brier_score_loss
import numpy as np


class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(enumerate(class_weights))

# Creating a LSTM model using the best hyperparameters from the SMOTE dataset for comparison
model = Sequential([
    Bidirectional(LSTM(units=best_params['model__units_1'], activation='tanh', return_sequences=True), 
                  input_shape=(1, X_train_scaled_non_smote.shape[1])),
    Dropout(best_params['model__dropout_1']),
    LSTM(units=best_params['model__units_2'], activation='tanh', return_sequences=False),
    Dropout(best_params['model__dropout_2']),
    Dense(units=32, activation='relu'),
    Dense(units=1, activation='sigmoid')  
])

optimizer = Adam(learning_rate=best_params['model__learning_rate'])
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

model.fit(X_train_reshaped_non_smote, y_train,
          validation_split=0.2,
          epochs=best_params['epochs'],
          batch_size=best_params['batch_size'],
          class_weight=class_weight_dict,
          callbacks=[lr_scheduler, early_stopping])

print("\n--- Final Model Evaluation ---")
X_test_reshaped = np.expand_dims(X_test_scaled_non_smote, axis=1)
y_test_pred = (model.predict(X_test_reshaped) > 0.5).astype(int)
y_test_proba = model.predict(X_test_reshaped)

X_train_reshaped = np.expand_dims(X_train_scaled_non_smote, axis=1)
y_train_pred = (model.predict(X_train_reshaped) > 0.5).astype(int)
y_train_proba = model.predict(X_train_reshaped)

print("\nTraining Set Metrics:")
print(classification_report(y_train, y_train_pred))
print("ROC AUC Score:", roc_auc_score(y_train, y_train_proba))
print("Brier Score:", brier_score_loss(y_train, y_train_proba))

print("\nTest Set Metrics:")
print(classification_report(y_test, y_test_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_test_proba))
print("Brier Score:", brier_score_loss(y_test, y_test_proba))



Epoch 1/20


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


179/179 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7813 - loss: 0.4323 - val_accuracy: 0.9196 - val_loss: 0.1537 - learning_rate: 0.0100
Epoch 2/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9604 - loss: 0.1297 - val_accuracy: 0.9745 - val_loss: 0.0800 - learning_rate: 0.0100
Epoch 3/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9655 - loss: 0.0732 - val_accuracy: 0.9399 - val_loss: 0.1667 - learning_rate: 0.0100
Epoch 4/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9714 - loss: 0.0501 - val_accuracy: 0.9304 - val_loss: 0.1919 - learning_rate: 0.0100
Epoch 5/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9759 - loss: 0.0427 - val_accuracy: 0.9769 - val_loss: 0.0708 - learning_rate: 0.0100
Epoch 6/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9940 - loss: 0.0143 - val_accuracy: 0.9598 - val_loss: 0.1071 - learning_rate: 0.0100
Epoch 7/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9858 - loss: 0.0251 - val_accurac

**LSTM Pseudolabelling using Non-SMOTE Data**

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.metrics import roc_auc_score, brier_score_loss, classification_report

model = Sequential([
    Bidirectional(LSTM(best_params['model__units_1'], activation='tanh', return_sequences=True), input_shape=(1, X_train_reshaped_non_smote.shape[2])),
    Dropout(best_params['model__dropout_1']),
    LSTM(best_params['model__units_2'], activation='tanh', return_sequences=False),
    Dropout(best_params['model__dropout_2']),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  
])

optimizer = Adam(learning_rate=best_params['model__learning_rate'])
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),  
    y=y_train  # Your training labels
)
class_weight_dict = dict(enumerate(class_weights))

augmented_train_data = X_train_dropped.values  

augmented_train_labels = y_train  

augmented_train_labels = np.reshape(augmented_train_labels, (-1, 1))  

augmented_train_data_reshaped = augmented_train_data.reshape((augmented_train_data.shape[0], 1, augmented_train_data.shape[1]))

model.fit(augmented_train_data_reshaped, augmented_train_labels, 
          epochs=best_params['epochs'], 
          batch_size=best_params['batch_size'],
          class_weight=class_weight_dict, 
          callbacks=[lr_scheduler, early_stopping])



confidence_threshold = 0.9  
n_iterations = 20  

for i in range(n_iterations):
    print(f"\n--- Pseudolabeling Iteration {i+1} ---")

    X_test_reshaped_no_smote = X_test_dropped.values.reshape((X_test_dropped.shape[0], 1, X_test_dropped.shape[1]))

    val_pred_proba = model.predict(X_test_reshaped_no_smote)

    pseudolabels = np.where(val_pred_proba.max(axis=1) > confidence_threshold, val_pred_proba.argmax(axis=1), -1)
    
    pseudolabeled_data = X_test_dropped.values[pseudolabels != -1]
    pseudolabel_targets = pseudolabels[pseudolabels != -1]

    if len(pseudolabeled_data) == 0:
        print("No high-confidence pseudolabels found. Stopping iteration.")
        break

    augmented_train_data = np.vstack([X_train_dropped, pseudolabeled_data])
    augmented_train_labels = np.concatenate([y_train, pseudolabel_targets])

    augmented_train_labels = np.reshape(augmented_train_labels, (-1, 1))  

    augmented_train_data_reshaped = augmented_train_data.reshape((augmented_train_data.shape[0], 1, augmented_train_data.shape[1]))

    model.fit(augmented_train_data_reshaped, augmented_train_labels, 
              epochs=best_params['epochs'], 
              batch_size=best_params['batch_size'],
              class_weight=class_weight_dict, 
              callbacks=[lr_scheduler, early_stopping])


print("\n--- Final Model Evaluation ---")

X_train_reshaped = X_train_dropped.values.reshape((X_train_dropped.shape[0], 1, X_train_dropped.shape[1]))

y_train_pred = (model.predict(X_train_reshaped) > 0.5).astype(int)  
y_train_proba = model.predict(X_train_reshaped)  

print("\nTraining Set Metrics:")
print(classification_report(y_train, y_train_pred))
print("ROC AUC Score:", roc_auc_score(y_train, y_train_proba))
print("Brier Score:", brier_score_loss(y_train, y_train_proba))


X_test_reshaped = X_test_dropped.values.reshape((X_test_dropped.shape[0], 1, X_test_dropped.shape[1]))

y_test_pred = (model.predict(X_test_reshaped) > 0.5).astype(int)
y_test_proba = model.predict(X_test_reshaped)  

print("\nTest Set Metrics:")
print(classification_report(y_test, y_test_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_test_proba))
print("Brier Score:", brier_score_loss(y_test, y_test_proba))



augmented_train_labels shape: (14303, 1)
augmented_train_data shape: (14303, 586)
augmented_train_data_reshaped shape: (14303, 1, 586)
augmented_train_labels shape: (14303, 1)
Epoch 1/20


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


224/224 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8753 - loss: 0.3856 - learning_rate: 0.0100
Epoch 2/20
 38/224 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9293 - loss: 0.1264

/opt/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/callback_list.py:151: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate
  current = self.get_monitor_value(logs)


224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9434 - loss: 0.1326 - learning_rate: 0.0100
Epoch 3/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9604 - loss: 0.0933 - learning_rate: 0.0100
Epoch 4/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9646 - loss: 0.0733 - learning_rate: 0.0100
Epoch 5/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9787 - loss: 0.0426 - learning_rate: 0.0100
Epoch 6/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9857 - loss: 0.0319 - learning_rate: 0.0100
Epoch 7/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9901 - loss: 0.0247 - learning_rate: 0.0100
Epoch 8/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9839 - loss: 0.0336 - learning_rate: 0.0100
Epoch 9/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9923 - loss: 0.0276 - learning_rate: 0.0100
Epoch 10/20
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9915 - loss: 0.0227 - learning_rate: 0.0100
Epoch 11/20
224/224 